# 8장 (구글 캘린더를 연동한) 스마트 일정 관리 프로젝트 

##  8.2.1 구글 클라이언트 라이브러리 설치하기
- 구글 캘린더 OpenAPI : https://developers.google.com/calendar/quickstart/python

- !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

## 8.2.2 구글 애플리케이션 등록 및 인증키 발급받기
- 구글 클라우드 콘솔 : https://console.cloud.google.com

## 8.2.4 네이버 애플리케이션 등록 및 인증키 발급받기
- 네이버 개발자 센터 :  https://developers.naver.com/main

## 8.3 사전 지식 쌓기

## 8.3.1 네이버 지역 검색 OpenAPI 사용하기
- https://developers.naver.com/main

In [ ]:
import requests

# 네이버 애플리케이션의 client_id와 client_secret 키 설정
headers = {
    "X-Naver-Client-Id" : "<네이버 애플리케이션의 Client ID를 입력하세요>",
    "X-Naver-Client-Secret" : "<네이버 애플리케이션의 Client Secret를 입력하세요>"
}

# 지역 검색 요청 파라미터 설정
query="국민대 맛집"
params = {
    "sort" : "comment",
    "query" : query,
    "display" : 3
}

# 지역 검색 URL과 요청 파라미터
naver_local_url = "https://openapi.naver.com/v1/search/local.json"

# 지역 검색 요청
res = requests.get(naver_local_url, headers=headers, params=params)
if res.status_code == 200:
    # 지역 검색 결과 확인
    places = res.json().get('items')
    print(places)

## 8.3.2 구글 캘린더 CRUD 연습하기
- https://developers.google.com/calendar/v3/reference/?apix=true

### 8.3.2.1 구글 캘린더 인증

In [ ]:
# 필요한 라이브러리
from google_auth_oauthlib.flow import InstalledAppFlow
# 구글 캘린더 API 서비스 객체 생성
from googleapiclient.discovery import build
import datetime

# 구글 클라우드 콘솔에서 다운받은 OAuth 2.0 클라이언트 파일 경로
creds_filename = 'res/smart_scheduler/google_token.json' 
# 사용 권한 지정
# https://www.googleapis.com/auth/calendar 캘린더 읽기/쓰기 권한
# https://www.googleapis.com/auth/calendar.readonly 캘린더 읽기 권한
SCOPES = ['https://www.googleapis.com/auth/calendar']

# 파일에 담긴 인증 정보로 구글 서버에 인증하기
# 새 창이 열리면서 구글 로그인 및 정보 제공 동의 후 최종 인증이 완료됩니다.
flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
creds = flow.run_local_server(port=0)

### 8.3.2.2 서비스 객체 생성

In [ ]:
### 객체 생성
service = build('calendar', 'v3', credentials=creds)

### 8.3.2.3 일정 생성(Create)

In [ ]:
today=datetime.datetime.today().strftime("%Y-%m-%d") # 일정을 생성할 날짜 YYYY-mm-dd 포맷 ex) today = "2020-10-18"

event = {
    'summary': '오늘 배워 오늘 쓰는 OpenAPI 수업', # 일정 제목
    'location': '서울특별시 성북구 정릉동 정릉로 77', # 일정 장소
    'description': 'OpenAPI 수업에 대한 설명입니다. 정말 재밌습니다.', # 일정 설명
    'start': { # 시작 날짜
        'dateTime': today + 'T09:00:00',
        'timeZone': 'Asia/Seoul',
    },
    'end': { # 종료 날짜
        'dateTime': today + 'T10:00:00',
        'timeZone': 'Asia/Seoul',
    },
    'attendees': [ # 참석자
        {'email': 'lpage@example.com'},
        {'email': 'sbrin@example.com'},
    ],
    'reminders': { # 알림 설정
        'useDefault': False,
        'overrides': [
            {'method': 'email', 'minutes': 24 * 60}, # 24 * 60분 = 하루 전 알림
            {'method': 'popup', 'minutes': 10}, # 10분 전 알림
        ],
    },
}

# calendarId : 캘린더 ID. primary이 기본 값입니다.
event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

### 8.3.2.4 일정 조회(Read)

In [ ]:
# 조회에 사용될 요청 변수 지정
calendar_id = 'primary' # 사용할 캘린더 ID
today = datetime.date.today().strftime("%Y-%m-%d") # 일정을 조회할 날짜 YYYY-mm-dd 포맷
time_min = today + 'T00:00:00+09:00' # 일정을 조회할 최소 날짜
time_max = today + 'T23:59:59+09:00' # 일정을 조회할 최대 날짜
max_results = 5 # 일정을 조회할 최대 개수
is_single_events = True # 반복 일정의 여부
orderby = 'startTime' # 일정 정렬

events_result = service.events().list(calendarId = calendar_id,
                                      timeMin = time_min,
                                      timeMax = time_max,
                                      maxResults = max_results,
                                      singleEvents = is_single_events,
                                      orderBy = orderby
                                      ).execute()

items = events_result.get('items')
print("===== [일정 목록 출력]=====")
print(items)

### 8.3.2.5 일정 수정(Update)

In [ ]:
event = events_result.get('items')[0]
event_id = event.get('id')

# 원하는 일정의 속성 값 변경하기
event['summary'] = "(수정된)" + event['summary']

# 일정 수정 요청하기
updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event).execute()

### 8.3.2.6 일정 삭제(Delete)

In [ ]:
# eventId : 일정을 조회한 후 얻은 id 값을 의미합니다.
eventId = updated_event.get('id')
service.events().delete(calendarId='primary', eventId=eventId).execute()

## 8.4 구현하기

## Step 1) 구글 캘린더 일정 조회하기

In [ ]:
### 필요한 라이브러리
from google_auth_oauthlib.flow import InstalledAppFlow
# 구글 캘린더 API 서비스 객체 생성
from googleapiclient.discovery import build
import datetime

# 구글 클라우드 콘솔에서 다운받은 OAuth 2.0 클라이언트 파일 경로
creds_filename = 'res/smart_scheduler/google_token.json' 
# 사용 권한 지정
# https://www.googleapis.com/auth/calendar 캘린더 읽기/쓰기 권한
# https://www.googleapis.com/auth/calendar.readonly 캘린더 읽기 권한
SCOPES = ['https://www.googleapis.com/auth/calendar']

# 파일에 담긴 인증 정보로 구글 서버에 인증하기
# 새 창이 열리면서 구글 로그인 및 정보 제공 동의 후 최종 인증이 완료됩니다.
flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
creds = flow.run_local_server(port=0)

### 객체 생성
service = build('calendar', 'v3', credentials=creds)

## Step 2) 구글 캘린더 일정 조회 및 데이터 정제하기

In [ ]:
# 조회에 사용될 요청 변수 지정
calendar_id = 'primary' # 사용할 캘린더 ID
today = datetime.date.today().strftime("%Y-%m-%d") # 일정을 조회할 날짜 YYYY-mm-dd 포맷
time_min = today + 'T00:00:00+09:00' # 일정을 조회할 최소 날짜
time_max = today + 'T23:59:59+09:00' # 일정을 조회할 최대 날짜
max_results = 5 # 일정을 조회할 최대 개수
is_single_events = True # 반복 일정의 여부
orderby = 'startTime' # 일정 정렬

# 오늘 일정 가져오기
events_result = service.events().list(calendarId = calendar_id,
                                      timeMin = time_min,
                                      timeMax = time_max,
                                      maxResults = max_results,
                                      singleEvents = is_single_events,
                                      orderBy = orderby).execute()

items = events_result.get('items')
print("===== [일정 목록 출력]=====")
print(items)
item = items[0] # 테스트를 위해 오늘 일정에서 한 개만 가져옵니다.

# 일정 제목
gsummary = item.get('summary')

# 일정 제목에서 [식사-국민대]에서 카테고리와 장소를 추출합니다.
gcategory, glocation = gsummary[gsummary.index('[')+1 : gsummary.index(']')].split('-')

# 구글 캘린더 일정이 연결되어있는 링크입니다.
gevent_url = item.get('htmlLink')

print("\n\n===== [일정 상세 정보 출력]=====")
print("category : ", gcategory)
print("location : ", glocation)
print("event_url : ", gevent_url)

## Step 3) 네이버 지역 검색으로 맛집 검색하기

In [ ]:
import requests

# 네이버 애플리케이션의 client_id와 client_secret 키 설정
headers = {
    "X-Naver-Client-Id" : "<네이버 애플리케이션의 Client ID를 입력하세요>",
    "X-Naver-Client-Secret" : "<네이버 애플리케이션의 Client Secret를 입력하세요>"
}

# 지역 검색 요청 파라미터 설정
query= glocation + " 맛집"
params = {
    "sort" : "comment",
    "query" : query,
    "display" : 3
}

# 지역 검색 URL과 요청 파라미터
naver_local_url = "https://openapi.naver.com/v1/search/local.json"

# 지역 검색 요청
res = requests.get(naver_local_url, headers=headers, params=params)

# 지역 검색 결과 확인
places = res.json().get('items')
print(places)

## Step 4) 카카오톡 메시지를 보내기 위한 사전 작업 준비하기

In [ ]:
import json
import kakao_utils

KAKAO_TOKEN_FILENAME = "res/kakao_message/kakao_token.json" 
KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>" 
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

## Step 5) 카카오톡 메시지 전송하기

In [ ]:
# 일정 주소 네이버 연결할 링크입니다.
gaddr_url = "https://search.naver.com/search.naver?query=" + glocation + " 맛집"
# contents 변수를 초기화 합니다.
contents = []

# 카카오톡 리스트 템플릿을 작성해봅니다.
template = {
    "object_type" : "list",
    "header_title" : gsummary + " - 맛집 추천",
    "header_link" : {
        "web_url": gevent_url,
        "mobile_web_url" : gevent_url
    },
    "contents" : contents,
    "buttons" : [
        {
            "title" : "일정 자세히 보기",
            "link" : {
                "web_url": gevent_url,
                "mobile_web_url" : gevent_url
            }
        },
        {
            "title" : "일정 장소 보기",
            "link": {
                "web_url": gaddr_url ,
                "mobile_web_url": gaddr_url
            }
        }
    ],
}


# 카카오톡 리스트 템플릿의 contents를 구성합니다.
for place in places:
    ntitle = place.get('title') # 장소 이름
    ncategory = place.get('category') # 장소 카테고리
    ntelephone = place.get('telephone') # 장소 전화번호
    nlocation = place.get('address') # 장소 지번 주소

    # 각 장소를 클릭할 때 네이버 검색으로 연결해주기 위해 작성된 코드
    query = nlocation + ' ' + ntitle

    # 장소 카테고리가 카페이면 카페 이미지
    # 이외에는 음식 이미지
    if '카페' in ncategory:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else:
        image_url = "https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill"

    # 전화번호가 있다면 제목과 함께 넣어줍니다.
    if ntelephone:
        ntitle = ntitle + "\ntel) " + ntelephone

    # 카카오톡 리스트 템플릿 형식에 맞춰줍니다.
    content = {
        "title": "[" + ncategory + "] " + ntitle,
        "description": ' '.join(nlocation.split()[1:]),
        "image_url": image_url,
        "image_width": 50, "image_height": 50,
        "link": {
            "web_url": "https://search.naver.com/search.naver?query=" + query,
            "mobile_web_url": "https://search.naver.com/search.naver?query=" + query
        }
    }
    contents.append(content)

# 카카오톡 메시지 전송
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('일정 맞춤 맛집을 성공적으로 보냈습니다.')
else:
    print('일정 맞춤 맛집을 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())